In [3]:
# Test Run
from Simulator import Simulator
import json

simulator = Simulator(edge_scale=10, 
                      replica_scale=1,
                      replica_size=1024, 
                      corruption_rate=0.1)
metrics = simulator.run()
print(json.dumps(metrics, indent=4))

08:28:23 Simulator: Simulation started.
08:28:23 Simulator: Parameter settings: 
                        edge_scale=10, 
                        replica_scale=1,
                        replica_size=1024bytes, 
                        corruption_rate=0.1
08:28:23 Simulator: Running cluster formation...
08:28:23 Simulator: Creating the rtt_matrix...
08:28:23 Simulator: RTT matrix created successfully!
08:28:23 Simulator: Creating the similarity_matrix...
08:28:23 Simulator: Parameter settings: 
                        failure_rate=0.02
08:28:23 Simulator: Similarity matrix created successfully!
08:28:23 Simulator: Clustering the servers...
08:28:23 Simulator: Servers clustered successfully!
08:28:23 Simulator: Clusters formed successfully!
08:28:23 Simulator: Initializing edge servers...


TypeError: EdgeServer.__init__() got an unexpected keyword argument 'data_replicas'

In [1]:
# Sampling Scale vs Time (average over 10 rounds for each scale)
n_rounds = 10
ns = [10, 20, 30, 40, 50]
l_times = {}
g_times = {}

for n in ns:
    l_times[n] = []
    g_times[n] = []
    for i in range(n_rounds):
        simulator = Simulator(edge_scale=n,
                              replica_scale=1,
                              data_replica_size=1024, 
                              corruption_rate=0.1)
        metrics = simulator.run()
        l_times[n].append(metrics['duration']['local'])
        g_times[n].append(metrics['duration']['global'])

with open('sampling_scale_vs_time.json', 'w') as f:
    json.dump({'local': l_times, 'global': g_times}, f)

NameError: name 'Simulator' is not defined

In [ ]:
# Replica Scale vs Time (sum over each replica scale)
rss = [10, 20, 30, 40, 50]
l_times = {}
g_times = {}

for rs in rss:
    l_times[rs] = []
    g_times[rs] = []
    for i in range(rs):
        simulator = Simulator(n=10, 
                              data_replica_size=1024, 
                              corruption_rate=0.1)
        metrics = simulator.run()
        l_times[rs].append(metrics['l_time'])
        g_times[rs].append(metrics['g_time'])

with open('replica_scale_vs_time.json', 'w') as f:
    json.dump({'local': l_times, 'global': g_times}, f)

In [ ]:
# Replica Size vs Time (average over 10 rounds for each replica size)
n_rounds = 10
rss = [10, 20, 30, 40, 50]
l_times = {}
g_times = {}

for rs in rss:
    l_times[rs] = []
    g_times[rs] = []
    for i in range(n_rounds):
        simulator = Simulator(n=10, 
                              data_replica_size=rs, 
                              corruption_rate=0.1)
        metrics = simulator.run()
        l_times[rs].append(metrics['l_time'])
        g_times[rs].append(metrics['g_time'])

with open('replica_size_vs_time.json', 'w') as f:
    json.dump({'local': l_times, 'global': g_times}, f)

In [ ]:
# Corruption Rate vs Time (average over 10 rounds for each replica size)
n_rounds = 10
crs = [0, 0.05, 0.1, 0.2, 0.3]
l_times = {}
g_times = {}

for cr in crs:
    l_times[cr] = []
    g_times[cr] = []
    for i in range(n_rounds):
        simulator = Simulator(n=10, 
                              data_replica_size=1024,
                              corruption_rate=cr)
        metrics = simulator.run()
        l_times[cr].append(metrics['l_time'])
        g_times[cr].append(metrics['g_time'])

with open('corruption_rate_vs_time.json', 'w') as f:
    json.dump({'local': l_times, 'global': g_times}, f)

In [ ]:
# local/global time consumption pie chart
# keep all parameters as fixed
l_times = []
g_times = []
for i in range(10):
    simulator = Simulator(n=10, 
                          data_replica_size=1024, 
                          corruption_rate=0.1)
    metrics = simulator.run()
    l_times.append(metrics['l_time'])
    g_times.append(metrics['g_time'])

with open('time_consumption.json', 'w') as f:
    json.dump({'local': l_times, 'global': g_times}, f)


In [ ]:
# corruption detection rate (we might not need this because our one can guarantee 100%)
    # with direct interact with app vendor
    # without clustering
    # with random clustering
    # with clustering
    # with clustering several rounds with reputation

# simulation vs real
# clustering vs no clustering


In [ ]:
# communication overhead by calculating data packet size manually
